# Lab 1 - Time Domain Sonar Lab

### Written by Miki Lustig and Frank Ong 2016
#### Edited, debugged, and ported to Raspberry Pi by Nick Antipa, Li-Hao Yeh, and Miki Lustig 2018
##### Updated by Alan Dong, Gautam Gunjala, and Josh Sans 2020

In this lab, we will interact with physical time-domain signals. The first part will involve generating and recording sounds on a laptop. We will use a linear chirp signal to characterize the response of the speaker-microphone system and look at detecting signals using cross-correlation.
In the second part, we will build on part one and use the speaker-microphone system to develop a simple sonar.


In [ ]:
# import functions and libraries
import numpy as np, matplotlib.pyplot as plt
from matplotlib.pyplot import *
from scipy import signal
from numpy import *
import sounddevice as sd

%matplotlib inline

## Part 1: Chirping!

For this assignment you will use the the Raspberry Pi equipped with a sound extension, a USB-powered speaker and a microphone. When playing a sound and recording, the signal goes through several systems. In particular it goes through the response of the sound extension output, the speaker, the room we are in, the response of the microphone, and the receive part of the sound extension.

A linear chirp is a signal in which the frequency increases linearly with time. In this assignment we will generate a chirp signal and use it to measure the amplitude of the frequency response of our speaker-room-microphone system. This lab will work best in a quiet environment. We recommend that you execute the lab at home or in a quiet place before submitting it.

An instantaneous frequency is defined as the derivative of the phase of a signal, $f = \frac{1}{2\pi} \frac{d\phi (t)}{ dt} $. For example, the instantaneous frequency of $\cos(\phi(t))=\cos(2\pi f_0 t)$ is  

$$f = \frac{d\phi (t)}{2\pi dt}  = f_0$$ 


For a linear chirp, the frequency changes linearly over time. The instantaneous frequency is therefore defined as 

<center>$$ f(t) = f_0 + kt. $$</center>


So,  

<center>$$ x(t) = \sin(2\pi\int_0^t f(t')dt') = \sin(2\pi\int_o^t(f_0+kt')dt') = \sin(2\pi(f_0+\frac{k}{2}t)t) $$</center>


### Part I, Task I: Generating the Chirp

Generate a 10 second long chirp signal, sampled at 48,000 Hz with a frequency range of 20 Hz to 20,000 Hz. Set the magnitude of the chirp to 0.5. This will help prevent non-linearities when we play the sound later. 

* Given $T$=total time length, $f_0$=start frequency, $f_1$ = end frequency, derive a formula $f(t)$ for the frequency sweep.
* Find the formula for the phase by integrating $\phi(t) = 2\pi\int_0^T f(t)dt$ to get the phase function.

Now, 
* Set the sample-rate frequency `fs = 48000` Hz
* Generate a time index from `t = 0` s to `t = 10` s with sampling rate of 48,000 Hz
* Generate a vector of phase vs time: `phi_of_t`  (  $\phi(t)$ )
* Generate the chirp function `s_chirp` with amplitude of 0.5 by plugging the phase into a sinusoid. 


In [ ]:
fs = 48000.0
f0 = 20.0
f1 = 20000.0
T = 10.0

# your code here:

# generate time index
t = r_[0:T:(1/fs)]

# generate chirp signal
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = 0.5*sin(phi_of_t)

* Plot the first 0.5 seconds of the chirp (`s_chirp`), you will notice that the carrier frequency increases and that the chirp has a constant envelope. To get a nice figure, make sure the aspect ratio of the figure is height/width = 0.2 . Label the axis and figure appropriately. 

In [ ]:
# set the aspect ratio such that the image is wide
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))

# your code here:
plt.plot(t[t<0.5], s_chirp[t<0.5])
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.title("Chirp signal")

* Plot the magnitude frequency response of the sequence from 0 to $\pi$ using the function `signal.freqz`. Note that the digital frequency range represents a physical frequency range of 0 Hz to 24,000 Hz. To get a nice figure, make sure the aspect ratio of the figure is height/width = 0.2. Label the axis and figure appropriately. 

The `signal.freqz` function on the Pi is very slow -- be patient. 

In [ ]:
# generate frequency response of chirp
w, S_chirp = signal.freqz(s_chirp, worN=512)

# generate frequency index
f = w/pi*fs/2 # convert from digital frequency to physical frequency
# f = r_[0:1:1/512] * fs/2

# plot
width, height = plt.figaspect(0.2)
fig = plt.figure(figsize=(width,height))
plt.plot(f, abs(S_chirp))
plt.xlabel("Frequency [Hz]")
plt.ylabel("Magnitude")
plt.title("Frequency response of chirp signal")

Why is the chirp is an appropriate signal to measure the magnitude frequency response of a system?


#### Answer:
A linear chirp sweeps all frequencies within its bandwidth. The definition of a frequency response is the response of a system to a constant frequency input. If the chirp is slow enough, then it could be considered to be a series of constant frequency inputs. By measuring the resulting (approximately) constant frequency outputs, we can obtain the system's frequency response!

### Part I, Task II: Playing and Recording the Chirp
Now, we will play the sound of the chirp on our speaker and simultaneously record using the microphone. 

__Playing and recording audio:__

* Run the following code. It is an example of how to play and record sound at the same time using the `sounddevice` package.

The resulting received sequence will be stored in the variable `rcv_chirp`.

In [ ]:
# set default sample rate and number of channels. 

sd.default.samplerate = 48000
sd.default.channels = 1
rcv_chirp = sd.playrec(s_chirp, fs, channels=1, blocking=False) # play the chirp

rcv_chirp = rcv_chirp.reshape((rcv_chirp.shape[0],)) # reshape the vector for proper transform later

In [ ]:
rcv_chirp.shape

* Plot the frequency response of the received sequence. 
* Also, plot the absolute value of the received signal. Plotting the absolute value (sort of) displays the envelope of the chirp. 

Label the figures and use an aspect ratio of height/width = 0.2.


In [ ]:
## plot chirp response

# generate frequency response of chirp
w, RCV_chirp = signal.freqz(rcv_chirp, worN=512)

# generate frequency indices
f = w/pi*fs/2

# generate time indices
t = r_[0.0:len(rcv_chirp)]/fs

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(f, abs(RCV_chirp))
plt.title("Frequency response of the transceived chirp")
plt.xlabel("Frequency [Hz]")

fig = figure(figsize=(width,height))
plt.plot(t, abs(rcv_chirp))
plt.title("Absolute value of transceived chirp");
plt.xlabel("Time [s]")

* Look at your results. What is the implicit assumption we are making in order to claim that the result is a frequency response? 
(HINT: consider the case when the chirp is very short)

#### Answer:
What is interesting is that the time response of the chirp is similar to the frequency response. This makes perfect sense, since for a chirp, frequency is dependent on time. The implicit assumption that we are making is that the time effects in the room are much faster than the sweep of the chirp, so for each frequency we get the approximate steady-state response. If the chirp was much faster, we would have seen echoes coming back from reflecting surfaces, which will show up at different times. Then the DTFT of the transceived chirp would include these echoes, thus interfering with the frequency response estimate.

### Part I, Task III: Envelope Detection with Hilbert Transform. 
The absolute value of the result "sort of" displays the envelope; however, it is still modulated by the (now rectified) frequency sweep carrier. If we write down the response, it can be expressed approximately as 

$$y[n] = |H[n]| \sin(2\pi (f_0 +k[n*T])nT + \angle H[n])$$

where $|H[n]|$ is the frequency response for the instantaneous frequency at the nth sample and $\angle H[n]$ is its phase response. 

The reason that it is only an approximation is that there is an inherent assumption that we do not look at transient effects, only the steady state effect for each frequency. This is a good approximation because our chirp is very slow compared to the propagation of sound in the room. 

One way to get the envelope $|H[n]|$ is to convert it to its analytic signal. The analytic signal $x_a(t)$ of signal $x(t)$ is:

$$x_a = F^{-1}(F(x)\cdot 2U) = x + j y$$

where $F$ is the Fourier transform, $U$ the unit step function,
and $y$ *is* the Hilbert transform of $x$. In other words, the negative half of the frequency spectrum is zeroed
out, turning the real-valued signal into a complex signal. This is similar to the question in HW2!

The analytic signal of the received chirp will then be: 

$$ y_a[n] = |H[n]|e^{j2\pi (f_0 +k[n*T])nT + \angle H[n]} $$

The envelope can be detected by taking the magnitude. 
(The analytic function of $y$ seems to have one more phase shift $\pi/2$ since it is a $\sin$ function.)

* Compute the analytic signal by using the function `signal.hilbert` and plot its absolute value. Note that the discrete Hilbert Transform is not perfect, since it uses FIR filtering. This will show up as ripple in the envelope.

* Label the figures and use an aspect ratio of height/width = 0.2

In [ ]:
# your lovely code here:
rcv_chirp_a = signal.hilbert(rcv_chirp)
t = r_[0.0:len(rcv_chirp_a)]/fs

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t, abs(rcv_chirp_a))
plt.xlabel("Time [s]")
plt.ylabel("Envelope Amplitude")
plt.title("Envelope of the transcieved chirp")

### Part I, Task IV: Autocorrelation Properties of the Chirp:

In Part II of the lab, we will be sending and receiving chirp pulses to estimate delays between the tranceived pulses. This is done by cross-correlating / matched filtering the received signal with the known chirp pulse to detect the echoes. In this task, we will investigate the correlation properties of the chirp.

A cross-correlation is defined as:

$$ R_{xy}[n] = \sum_{m=-\infty}^\infty x[m]y^*[m-n] = (x[m]*y^*[-m])[n]$$

where $y^*[-m]$ is the complex conjugate of $y[-m]$. This is similar to a convolution, but without flipping one of the signals. It can be implemented using a convolution as shown above. In general, the more correlated the two signals are at position $n$, the higher the value will be. That's why it is useful in a sonar system.

#### Matched Filter 
When we look for a very specific shape in a signal, we can compute a cross-correlation between the signal and the shape we are interested in. In that case, the operation of the cross-correlation is also called a matched filter -- i.e. correlating with a filter that is matched to the shape we look for.

Because we will be doing cross-correlations between a chirp pulse and its echoes, it is useful to look at the autocorrelation, which is basically a cross-correlation of the signal with itself. A discrete autocorrelation of a signal is defined as: 

$$ R_{xx}[n] = \sum_{m=-\infty}^\infty x[m]x^*[m-n] = (x[m]*x^*[-m])[n]$$ 

The chirp has a very nice property that its autocorrelation is very narrow. Since the spread of the resulting correlation determines the time resolution of detection, the width of the autocorrelation is important. This property is called pulse compression and is widely considered in radar design. Random noise and some other pseudo-random sequences also possess this property.

* Generate a 512 sample chirp pulse with a frequency sweep from 17 kHz to 18 kHz and sampling rate fs = 48000 Hz. 
* Validate its frequency response by plotting it.

In [ ]:
# your beautiful code here:
fs = 48e3
N = 512
T = N/fs
t = r_[0.0:N]/fs
f0 = 17e3
f1 = 18e3

# generate chirp signal
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t)

# generate frequency response of chirp
w, S_chirp = signal.freqz(s_chirp, worN=512)

# generate frequency index
f = w/pi*fs/2

# plot
width, height = figaspect(0.1)
fig = figure(figsize=(width,height))
plt.plot(f, abs(S_chirp))
plt.xlabel("Frequency [Hz]")
plt.ylabel("Magnitude")
plt.title("Frequency response of chirp")


* Compute the autocorrelation of the chirp using discrete convolution, either with `signal.convolve` or `signal.fftconvolve`. Remember that you have to flip the signal since convolution does that already. You can flip a signal `x` by doing `x[::-1]`. Use `mode="full"` for convolution.
* Plot the autocorrelation. Your plot should be spiky because we did not do envolope detection yet. Use miliseconds for the x-axis.

In [ ]:
# your fantastic code here:

# compute autocorrelation
chirp_acorr = signal.convolve(s_chirp, conj(s_chirp[::-1]), mode="full")
t = (r_[-len(chirp_acorr)/2.0:len(chirp_acorr)/2.0] + 0.5) / fs * 1e3

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t, chirp_acorr)
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude")
plt.title("Autocorrelation of chirp")

In a similar way as we did before, it is possible to recover the envelope of the autocorrelation by performing the cross-correlation with the analytic signal and then taking the absolute value. In this case, we know exactly what the analytic function is!

* Generate `s_chirp_a`, the analytic function of the chirp by computing: `s_chirp_a = exp(1j*phi_of_t)`. Perform the cross-correlation between `s_chirp_a` and `s_chirp` and show the envelope. As stated before, this could also be called a matched filter. 
* Measure the full-width at half max (FWHM) of the main lobe of the autocorrelation. 
* Comment on the FWHM of the main lobe of the matched filter with respect to the duration of the pulse. That ratio is also called pulse compression.  For simplicity, normalize the plot such that the maximum is 1, but record the maximum value of the autocorrelation and display it in the title of the figure.  

Use the pragma ``%matplotlib notebook`` for making the figure interactive, and ``plt.grid('on')`` for displaying a grid.

Use milliseconds for the x-axis.



In [ ]:
%matplotlib notebook
%matplotlib notebook

# your nice script to produce beautiful chirps, cross-correlations, and figures here:

# compute autocorrelation and envelope
s_chirp_a = exp(1j*phi_of_t)
chirp_acorr = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))
max_chirp_acorr = max(abs(chirp_acorr))
tt = (r_[-len(chirp_acorr)/2.0:len(chirp_acorr)/2.0] + 0.5) / fs * 1e3

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(tt, abs(chirp_acorr)/max_chirp_acorr)
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Autocorrelation envelope of chirp, max=%f" %max_chirp_acorr)
plt.grid(True)

#### Your answer here:
The main lobe of the matched filter has a FWHM of ~1.23 ms while the original pulse has a duration of ~10.67 ms. Effectively, the pulse is compressed!

Now we will look at why the chirp pulse is better for cross-correlation detection than a pure tone.
- Repeat Task IV for:
 1. A constant frequency of 17,000 Hz, 512 samples in length. 
 2. A chirp with a frequency sweep from 16,500 Hz to 17,500 Hz (1 kHz bandwidth), 512 samples in length.  
 3. A chirp with a frequency sweep from 15,000 Hz to 19,000 Hz (4 kHz bandwidth), 512 samples in length.
- Compare the widths of the main lobes (full width at half max). How much "Pulse Compression" are you getting by using a chirp for detection compared to a single frequency pulse?
- What is the approximate bandwidth of the pure frequency pulse and what is the bandwidth of the chirp pulses? Comment on the tradeoff between bandwidth and pulse compression.
- What is the maximum autocorrelation for each pulse?
   

In [ ]:
# your brilliant code here:
fs = 48e3
N = 512
T = N/fs
t = r_[0.0:N]/fs

# generate pure tone
f0 = 17e3
s_tone = sin(2*pi*f0*t)
s_tone_a = exp(1j*2*pi*f0*t)
cross_corr_1 = signal.convolve(s_tone, conj(s_tone_a[::-1]))
tt = (r_[-len(cross_corr_1)/2.0:len(cross_corr_1)/2.0] + 0.5) / fs * 1e3

# generate lower bandwidth chirp signal
f0 = 16.5e3
f1 = 17.5e3
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t)
s_chirp_a = exp(1j*phi_of_t)
cross_corr_2 = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))

# generate higher bandwidth chirp signal
f0 = 15e3
f1 = 19e3
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t)
s_chirp_a = exp(1j*phi_of_t)
cross_corr_3 = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(tt, abs(cross_corr_1)/max(abs(cross_corr_1)))
plt.plot(tt, abs(cross_corr_2)/max(abs(cross_corr_2)))
plt.plot(tt, abs(cross_corr_3)/max(abs(cross_corr_3)))
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Effect of pulse bandwidth on autocorrelation envelope")
plt.legend(("Tone, max=%i" %max(abs(cross_corr_1)), "1 kHz chirp, max=%i" %max(abs(cross_corr_2)), "4 kHz chirp, max=%i" %max(abs(cross_corr_3))))
plt.grid(True)


#### Answer and interpretations here:
The FWHM of the autocorrelation envelope of the constant frequency signal is \~10.66 ms, the same as its pulse duration. The main lobe for the 1 kHz bandwidth chirp has a FWHM of \~1.23 ms and the 4 kHz chirp has a FWHM of \~0.30 ms. Their pulse compression ratios are ~8.67 and ~35.3, respectively.
At the same time, the bandwidth of the constant frequency pulse is approximately 1/(its duration) = ~94 Hz, and the bandwidths of the chirps are 1 kHz and 4 kHz -- these are inversely proportional to the main lobe widths!
So, you can't be compact in one domain without expanding in the other...



- Now, repeat Task IV for a chirp with a frequency sweep from 16,500 Hz to 17,500 Hz, 256 samples in length
- Compare the width of the main lobe (full width at half max) to the previous case of 16,500 Hz - 17,500 Hz, 512 samples in length.
- Compare the maximum autocorrelation values as well.

What's the effect of having more bandwidth? What's the effect of having longer/shorter pulses?


In [ ]:
# your amazing code here:
fs = 48e3
N = 256
T = N/fs
t = r_[0.0:N]/fs

# generate chirp signal
f0 = 16.5e3
f1 = 17.5e3
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t)
s_chirp_a = exp(1j*phi_of_t)
cross_corr_4 = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))
tt2 = (r_[-len(cross_corr_4)/2.0:len(cross_corr_4)/2.0] + 0.5) / fs * 1e3

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(tt2, abs(cross_corr_4)/max(abs(cross_corr_4)))
plt.plot(tt, abs(cross_corr_2)/max(abs(cross_corr_2)))
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Effect of pulse duration on autocorrelation envelope")
plt.legend(("256-length 1 kHz chirp, max=%i" %max(abs(cross_corr_4)), "512-length 1 kHz chirp, max=%i" %max(abs(cross_corr_2))))
plt.grid(True)

####  Your answer below:
These two chirp pulses have the same FWHM, but the one that's twice as long has twice the peak autocorrelation value. The main lobe width is proportional to the pulse bandwidth. The peak correlation value is proportional to the pulse duration.

### Dealing with sidelobes
As you can see, the chirp provides good pulse compression of the main lobe. However, there exist very strong sidelobes. This is because the chirp is multiplied with a rect function; that is abrupt. Instead, we will window the chirp with one of the smooth window functions to taper off the sidelobes.

* Repeat the above for a 512-length chirp with a sweep from 16.5 kHz to 17.5 kHz, and from 15 kHz to 19 kHz. This time, multiply the chirp (and its analytic function) with a Hann window. You will find the function `signal.hann` useful. 

* Plot the normalized autocorrelations (in the same figure).
* Comment on the magnitude of the sidelobes.
* Comment on the width of the main lobes.
* What's the tradeoff?


In [ ]:
# your code here:
fs = 48e3
N = 512
T = N/fs
t = r_[0.0:N]/fs

# generate lower bandwidth chirp signal
f0 = 16.5e3
f1 = 17.5e3
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t) * signal.hann(N)
s_chirp_a = exp(1j*phi_of_t) * signal.hann(N)
cross_corr_5 = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))
tt = (r_[-len(cross_corr_5)/2.0:len(cross_corr_5)/2.0] + 0.5) / fs * 1e3

# generate higher bandwidth chirp signal
f0 = 15e3
f1 = 19e3
k = (f1-f0)/T
phi_of_t = 2*pi*(f0 + k/2*t)*t
s_chirp = sin(phi_of_t) * signal.hann(N)
s_chirp_a = exp(1j*phi_of_t) * signal.hann(N)
cross_corr_6 = signal.convolve(s_chirp, conj(s_chirp_a[::-1]))

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(tt, abs(cross_corr_5)/max(abs(cross_corr_5)))
plt.plot(tt, abs(cross_corr_6)/max(abs(cross_corr_6)))
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Effect of windowing on autocorrelation envelope")
plt.legend(("Windowed 1 kHz chirp, max=%i" %max(abs(cross_corr_5)), "Windowed 4 kHz chirp, max=%i" %max(abs(cross_corr_6))))
plt.grid(True)

#### Answer:
The widths of the main lobes are approximately doubled. The FWHM for the windowed 1 kHz chirp is ~2.49 ms compared to ~1.23 ms for the unwindowed one. Likewise, the FWHM for the windowed 4 kHz chirp is ~0.65 ms compared to ~0.30 ms. Furthermore, the maximum autocorrelation values are reduced. But, the sidelobes are significantly smaller!

### You are now ready to proceed to the Sonar Lab!

## Part II: Sonar

In this part of the lab we will write a simple application that implements a speaker and microphone.

The basic idea is very simple and is the basis of sonar and ultrasound imaging -- objects reflect sound waves. If we send a pulse of sound, we will get reflection echoes of that pulse. Detecting the echoes and their time-of-flight will reveal their distance from the source, based on the speed of sound in air. 

The way we are going to implement the sonar is to generate a series of rapid pulses, and then use matched filtering to detect the returning echoes. There are many parameters in this lab that can be tweaked to get different results. We encourage you to experiment. We very much enjoyed making the lab and played quite a bit! We hope you enjoy it too. 

#### Instructions:
The microphone and speaker you have are somewhat directional. Make sure that the microphone and speaker point in the same direction. You will get the best quality in a quiet room, without interference from other sources of noise!

If you are getting poor results, please consult with us. 

This lab was inspired from an iPhone app called active-radar. 


### Part II, Task I: Generating Chirp Pulses

Recall from Part I, that the width of the main lobe of the autocorrelation depends on the bandwidth of the pulse. 
For a constant frequency pulse (i.e. a pure tone), the bandwidth will be inversly proportional to its length. Short pulses are localized in time, and therefore we will be able to separate echoes from targets that are closely spaced. However, short pulses carry less energy (for the same amplitude) and this will reduce our signal-to-noise ratio (SNR) in the detection and reduce our ability to detect the targets at all. So, in summary: for constant frequency pulse, there's an inherent tradeoff between the resolution of the sonar (distinguish between close targets) and the signal-to-noise ratio. 

If we use a chirp pulse, we can increase the length of the pulse while also increasing the bandwidth. This will enable us to improve our SNR as well as keep the resolution of our sonar (by preserving a large bandwidth).

In our implemetation we are going to design a pulsed sonar system in which we repeatedly send pulses and then listen to the returning echoes. The arrival time of the echoes will correspond to double the time-of-flight of sound propagation from our system to the target. 

* Write a function that generates a chirp pulse:
`pulse = genChirpPulse(Npulse, f0, f1, fs)`.

The function will accept: `Npulse` = number of samples, `f0, f1` = starting and ending frequency and `fs` = sampling rate. The function will return the analytic function of the chirp $\exp (j 2\pi \int_0^t f(t)dt )$ with amplitude 1.

In [ ]:
def genChirpPulse(Npulse, f0, f1, fs):
    # Function generates an analytic function of a chirp pulse
    # Inputs:
    #     Npulse = pulse length in samples
    #     f0     = starting frequency of chirp
    #     f1     = end frequency of chirp
    #     fs     = sampling frequency
    # Output:
    #     pulse  = chirp pulse
    
    T = Npulse/fs
    t = np.r_[0.0:Npulse]/fs
    k = (f1-f0)/T
    pulse = exp(1j*2*pi*(f0 + k/2*t)*t)
    
    return pulse


* To validate that the function works display the pulse generated with `Npulse = 200`, `f0 = 1000`, `f1 = 8000`, `fs = 48000`. Remember the pulse is complex, so plot the real and imaginary parts separately.

In [ ]:
%matplotlib inline
%matplotlib inline

# your code here:
pulse = genChirpPulse(200, 1000.0, 8000.0, 48000.0)
t = r_[0.0:len(pulse)]/fs * 1e3

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t, pulse.real)
plt.plot(t, pulse.imag)
plt.title("Chirp signal")
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude")

__Generate Pulse Trains__

Next, we will use the pulse generated by `genChirpPulse` and generate a pulse train.

* Write a new function `ptrain = genPulseTrain(pulse, Nrep, Nseg)`
that accepts `pulse` = a pulse generated by `genChirpPulse`,  `Nrep` = number of pulse repetitions, and `Nseg` = length of each pulse train segment (which is >= to the length of `pulse`).

The function returns `ptrain` which is a vector of length `Nrep * Nseg`. (Hint: use `np.tile`.)

In [ ]:
def genPulseTrain(pulse, Nrep, Nseg):
    # Function generates a pulse train from a pulse. 
    # Inputs:
    #     pulse  = the pulse generated by genChirpPulse
    #     Nrep   = number of pulse repetitions
    #     Nseg   = length of pulse segment >= len(pulse)
    # Output:
    #     ptrain = pulse train
    
    seg = zeros(Nseg)
    seg[:len(pulse)] = pulse
    ptrain = tile(seg, Nrep)
    
    return ptrain

### Part II, Task II: Record Echoes of Chirp Pulse Train

We now have components to generate pulses, generate a pulse train, and play and record it. Let's see what we get!
We will start with very short pulses with a single carrier frequency. Rectangular pulses are difficult for the speaker
to produce as they exhibit discontinuities at the beginning and the end of the pulse. Therefore we will multiply the pulses
with a smooth window. Here, we will use a Hann window.

* Generate a `f0` = `f1` = 8 kHz, `Npulse` = 96 pulse with `fs` = 48000. Window the pulse with a Hann window. This will result in a pulse length of 2 ms. You should be able to hear this tone.
* Plot the real and imaginary parts of the pulse.

In [ ]:
fs = 48e3
f0 = 8e3
f1 = 8e3
Npulse = 96

# your code here:
pulse_a = genChirpPulse(Npulse, f0, f1, fs) * signal.hann(Npulse)
pulse = pulse_a.real
t = r_[0.0:len(pulse)]/fs * 1e3

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t, pulse)
plt.plot(t, pulse_a.imag)
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.title("Windowed chirp")
plt.legend(('real','imag'))

* Use the real part of the pulse to generate a pulse train of `Nrep` = 15 pulses, `Nseg` = 4096 samples.

In [ ]:
# your code here:
Nrep = 15
Nseg = 4096
ptrain = genPulseTrain(pulse, Nrep, Nseg)

* Play and record the pulse train. Scale the amplitude of the pulses to 0.5. Make sure your volume is set to a maximum of 70% and look at the plot with the input pulse train and the received pulse train.

Use the pragma ``%matplotlib notebook`` for interactive plots, so you can zoom in on the result.

In [ ]:
#rcv = sd.playrec(ptrain/2.0, fs, channels=1, blocking=True)
rcv = sd.playrec(1000*ptrain, fs, channels=1, blocking=True) 
#You may have to chance the scalar value of 1000 above to adjust the loudness (increase for louder, decrease for softer)

rcv = rcv.reshape((rcv.shape[0],))

In [ ]:
%matplotlib notebook
%matplotlib notebook

# your code here:

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(ptrain)
plt.plot(rcv)
plt.title("Transceived pulse train")
plt.legend(("Transmitted", "Received"))

* Extract a single pulse from the received pulse train. You can find the starting index of the pulse from the interactive plot. Extract at least `2*Npulse` samples before the starting index and `20*Npulse` samples after using `rcv_pulse = rcv[idx-2*Npulse:idx+Npulse*20]`

* Plot the received pulse. Can you see any echoes?

You can disable interactivity using the pragma ``matplotlib inline``.


In [ ]:
%matplotlib notebook
%matplotlib notebook

# your code here:

# find index of start pulse
ind = argmax( abs(rcv[100::]) > 0.75*max(abs(rcv[100::])) ) + 100
inds = r_[ind-2*Npulse:ind+20*Npulse]
rcv_pulse = rcv[inds]
t = r_[0.0:len(rcv)]/fs * 1e3

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t[inds], rcv_pulse)
plt.xlabel("Time [ms]")
plt.ylabel("Amplitude")
plt.title("Single transceived pulse (plus echoes)")

#### Matched Filtering

The strong pulses we see are a result of direct feedthrough from the transmitter to the receiver that do not scatter off targets. The echoes we see are a result of echoes from reflecting surfaces. The problem in our setup is that we don't know the exact delay between the transmitter and the receiver. Instead, we will assume that the travel time for sound between the speaker and the microphone is negligible and much smaller than that of reflective targets. We can then detect when the pulses start based on the direct feedthrough signal. This assumption is very good as long as your speaker is close to the microphone!

We will detect both the feedthrough and echoes using matched filtering.

* Write a function `Xrcv = crossCorr(rcv, pulse_a)` to calculate the cross-correlation (matched filter) of the received signal with the analytic function of the pulse.  You can use `signal.convolve` or `signal.fftconvolve`.
* Take the absolute value of `Xrcv` to recover its envelope. Call the result `Xrcv_a`.

Make sure the plot is interactive with ``matplotlib notebook``.

In [ ]:
def crossCorr(rcv, pulse_a):
    # Funtion generates cross-correlation between rcv and pulse_a.
    # Inputs:
    #     rcv     = received signal
    #     pulse_a = analytic pulse
    # Output:
    #     Xrcv    = cross-correlation between rcv and pulse_a
    
    Xrcv = signal.convolve(rcv, pulse_a[::-1])
    
    return Xrcv

In [ ]:
%matplotlib notebook
%matplotlib notebook

# your code here:
Xrcv_a = abs(crossCorr(rcv, pulse_a))
t = r_[0.0:len(Xrcv_a)]/fs * 1e3

# plot
width, height = figaspect(0.3)
fig = figure(figsize=(width,height))
plt.plot(t, Xrcv_a)
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Cross-correlation of transceived pulse train")

* Again, extract a single pulse from the received pulse train using the same index. Extract at least `2*Npulse` samples before the pulse and `20*Npulse` samples after. Plot the received pulse. Can you see any echoes?

In [ ]:
%matplotlib notebook
%matplotlib notebook

# find index of start pulse
Xrcv_pulse = Xrcv_a[inds]
t = r_[0.0:len(Xrcv_a)]/fs * 1e3

# plot
width, height = figaspect(0.2)
fig = figure(figsize=(width,height))
plt.plot(t[inds], Xrcv_pulse)
plt.xlabel("Time [ms]")
plt.ylabel("Envelope Amplitude")
plt.title("Cross-correlation of single transceived pulse (plus echoes)")

#### Sonar System

In order to automate the system and visualize the results we need a few more components. To extract the pulses we need to know the position of the first feedthrough pulse. 


* Write a function `ind = findDelay(Xrcv_a, Nseg)` that takes the result of the matched filter and finds the index of the first feedthrough pulse. Try testing on the actual signal to check whether the function is correct. There are multiple ways of doing it. `Nseg` is not necessarily required.

In [ ]:
def findDelay(Xrcv, Nseg):
    # Function finds the first pulse index.
    # Inputs:  
    #     Xrcv = the received matched filtered signal
    #     Nseg = length of a segment
    # Output:
    #     ind  = index of the beginning of the first pulse
    
    ind = argmax(Xrcv[0:Nseg])
    
    return ind

In [ ]:
ind = findDelay(Xrcv_a, Nseg)
print(ind)

We now can correct for delays and detect echoes. The only thing left now is to convert the time between echoes into actual distance.

If we send a pulse of sound, we will get reflection echoes of that pulse. Detecting the echoes and their time-of-flight will reveal their distance from the source, based on the speed of sound in air. The speed of sound in air is given by the following equation:

$$ v_s = 331.5\sqrt{1+T/273.15}~\mathrm{m/s}$$ 

where T is the temperature in degrees Celcius. 

* Create a function `t = dist2time(dist, temperature)` that takes in the distance to the target in cm and converts it into the time in seconds between the transmitted pulse and its echo. Remember the arrival time includes the time to the target and back and therefore the time should be doubled. 
For example, for `temperature` = 20 degrees Celsius and `dist` = 400 cm, the time it takes is 0.023 s.

* Create a function `dist = time2dist(t, temperature)` that takes in the time to the target in seconds and converts it into the distance in cm between the transmitted pulse and its echo. Remember the arrival time includes the time to the target and back and therefore the time should be halved. 

In [ ]:
def dist2time(dist, temperature=21.0):
    # Function converts distance in cm to time in seconds.
    # Inputs:
    #     dist        = distance to object in cm
    #     temperature = in Celcius
    # Output:
    #     t           = time in seconds between transmitted pulse and echo
    
    v_sound = 331.5*sqrt(1 + temperature/273.15) * 100 # in cm/s
    t = dist/v_sound*2
    
    return t

def  time2dist(t, temperature=21.0):
    # Function converts time in seconds to distance in cm.
    # Inputs:
    #     t           = time in seconds between transmitted pulse and echo
    #     temperature = in Celcius
    # Output:
    #     dist        = distance to object in cm
    
    v_sound = 331.5*sqrt(1 + temperature/273.15) * 100 # in cm/s
    dist = t/2*v_sound
    
    return dist

## A sonar (almost)

* The following function will use your functions to generate pulses and display the matched filtering of each pulse as intensity of a horizontal line in an image. If nothing is moving, you will be able to see constant vertical lines representing echoes. If something is moving, you will be able to track the object's distance.  



In [ ]:
def sortOfASonar(Npulse, f0, f1, fs, Nrep, Nseg):
    pulse_a = genChirpPulse(Npulse, f0, f1, fs) * signal.hann(Npulse)
    pulse = pulse_a.real
    ptrain = genPulseTrain(pulse, Nrep, Nseg)
    rcv = sd.playrec(ptrain/2.0, fs, channels=1, blocking=True)
    rcv = rcv.reshape((rcv.shape[0],))
    Xrcv_a = abs(crossCorr(rcv, pulse_a))
    
    ind = findDelay(Xrcv_a, Nseg) 
    img = np.zeros((Nrep-1,Nseg))
    img[0,:] = Xrcv_a[ind:ind+Nseg]
    
    # Look for peak in each pulse in the pulse train to avoid drift between transmit and receive
    for n in range(1, Nrep-1):
        ind2 = findDelay(Xrcv_a[ind+Nseg//2:ind+Nseg//2+Nseg], Nseg)
        ind = ind + ind2 + Nseg//2
        img[n,:] = Xrcv_a[ind:ind+Nseg]
    
    return img

Now, use the function above to:  

* Generate a pulse train of 100 pulses. Each (Hann windowed) pulse should be length `Npulse = 72` samples (1.5 ms) and a constant frequency of 8 kHz. The spacing between pulses should be 0.1 seconds (`Nseg = 4800`). 
* Display the image.

In [ ]:
Npulse = 72
f0 = 8e3
f1 = 8e3
fs = 48e3
Nrep = 100
Nseg = 4800
img = sortOfASonar(Npulse, f0, f1, fs, Nrep, Nseg)

Display the result. Pay attention to the width of the echoes. 

In [ ]:
%matplotlib notebook
%matplotlib notebook

# display up to 2.5 m, approximately 700 samples at 48000 Hz sampling rate (and 21 degrees C)

vmax = 0.25 # threshold -- lower will be able to see smaller echoes

plt.imshow(img[:,0:700]/max(img.ravel()), vmax=vmax, aspect=10, cmap='gray', interpolation='bilinear', extent=(0,time2dist(700/fs), Nrep*Nseg/fs, 0))
plt.xlabel('Distance [cm]')
plt.ylabel('Time [s]')

Now, repeat the experiment with a chirp length of `Npulse = 360` samples, and a frequency sweep from 6 kHz to 12 kHz.
Pay attention to the resolution of the lines. 

In [ ]:
Npulse = 360
f0 = 6000.0
f1 = 12000.0
fs = 48000.0
Nrep = 100
Nseg = 4800
img = sortOfASonar(Npulse, f0, f1, fs, Nrep, Nseg)

In [ ]:
%matplotlib notebook
%matplotlib notebook

# display up to 2.5 m, approximately 700 samples at 48000 Hz sampling rate (and 21 degrees C)

vmax = 0.25 # threshold -- lower will be able to see smaller echoes

plt.imshow(img[:,0:700]/max(img.ravel()), vmax=vmax, aspect=10, cmap='gray', interpolation='bilinear', extent=(0,time2dist(700/fs), Nrep*Nseg/fs, 0))
plt.xlabel('Distance [cm]')
plt.ylabel('Time [s]')

Feel free to repeat while moving a target -- can you see the echoes changing? Try playing with different parameters. 




## A Real (Time) Sonar
You now have a working sonar! It would be much easier though to play with different parameters if we automate things, so we created some wrappers for real-time plotting in a separate notebook (lab1-RealTime-Sonar). 

* Copy-and-paste the 5 functions you created, including `genPulseTrain()`, `genChirpPulse()`, `crossCorr()`, `findDelay()`, and `dist2time()`, to the specified code cell in the Real-Time Sonar Lab.


### You are now ready to proceed to the Real-Time Sonar Lab!